<a href="https://colab.research.google.com/github/r-autowired/AIMLWorks/blob/main/Notes/Hackathons/UT_Hackathons/HK_2_BankLoanDefault_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [475]:
# Installing all required imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold

# importing metrics
from sklearn.metrics import confusion_matrix,roc_curve,classification_report,recall_score, f1_score,accuracy_score,precision_score

import random
import time

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor

# importing different functions to build models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import L2

# importing SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [476]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [477]:
df = pd.read_csv('/content/drive/MyDrive/AIML_UT/Colab_Files/Hackathons/UT-Hackathon-2/Train_set.csv')
df.head()

,ID,loan_amnt,loan_term,interest_rate,loan_grade,loan_subgrade,job_experience,home_ownership,annual_income,income_verification_status,...,delinq_2yrs,public_records,revolving_balance,total_acc,interest_receive,application_type,last_week_pay,total_current_balance,total_revolving_limit,default
0,72199369,9000,3 years,9.17,B,B2,<5 Years,OWN,85000.0,Not Verified,...,0.0,0.0,39519,20.0,59.60,INDIVIDUAL,4.0,95493.0,84100.0,0
1,14257956,18000,3 years,13.65,C,C1,<5 Years,OWN,64000.0,Verified,...,0.0,1.0,9783,24.0,3348.25,INDIVIDUAL,95.0,185433.0,13500.0,0
2,66216451,16000,3 years,7.26,A,A4,<5 Years,MORTGAGE,150000.0,Source Verified,...,2.0,0.0,13641,27.0,276.69,INDIVIDUAL,13.0,180519.0,19300.0,0
3,46974169,25000,3 years,13.99,C,C4,NaN,MORTGAGE,59800.0,Verified,...,0.0,0.0,35020,35.0,1106.72,INDIVIDUAL,17.0,183208.0,55400.0,0
4,46725961,17000,3 years,6.39,A,A2,10+ years,MORTGAGE,72000.0,Source Verified,...,0.0,0.0,23990,26.0,725.29,INDIVIDUAL,39.0,23990.0,81300.0,0


In [478]:
def commonMethodTrain(df):
  # Calculate the mode of 'job_experience' and fill missing values
  job_experience_mode = df['job_experience'].mode()[0]
  df['job_experience'].fillna(job_experience_mode, inplace=True)

  # Calculate the median for the specified numerical columns and fill missing values
  numerical_cols_to_impute = ['annual_income', 'delinq_2yrs', 'public_records', 'total_acc', 'last_week_pay', 'total_current_balance', 'total_revolving_limit']
  for col in numerical_cols_to_impute:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

  # Verify that there are no more missing values in the imputed columns
  print(df[numerical_cols_to_impute + ['job_experience']].isnull().sum())


  # Analyze each categorical column and determine the encoding strategy

  # Columns identified as object type:
  # ['loan_term', 'loan_grade', 'loan_subgrade', 'job_experience', 'home_ownership', 'income_verification_status', 'loan_purpose', 'state_code', 'application_type']

  # Analyze 'loan_term': Check unique values and if ordinal
  print(df['loan_term'].value_counts())
  # This is likely nominal or ordinal (shorter term vs longer term). Given the limited unique values, One-Hot Encoding is suitable.

  # Analyze 'loan_grade': Check unique values and if ordinal
  print(df['loan_grade'].value_counts())
  # This is ordinal (A is better than B, etc.). Label Encoding is suitable.

  # Analyze 'loan_subgrade': Check unique values and if ordinal
  print(df['loan_subgrade'].value_counts())
  # This is ordinal (A1 is better than A2, etc.). Label Encoding is suitable, but it has many unique values, so One-Hot Encoding might also be considered depending on the model. Let's start with Label Encoding and re-evaluate if necessary.

  # Analyze 'job_experience': Check unique values and if ordinal
  print(df['job_experience'].value_counts())
  # This is ordinal (e.g., 10+ years is more than <5 years). Label Encoding is suitable.

  # Analyze 'home_ownership': Check unique values and if ordinal
  print(df['home_ownership'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'income_verification_status': Check unique values and if ordinal
  print(df['income_verification_status'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'loan_purpose': Check unique values and if ordinal
  print(df['loan_purpose'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'state_code': Check unique values and if ordinal
  print(df['state_code'].value_counts())
  # This is nominal and has many unique values. One-Hot Encoding will result in a very wide matrix. Consider alternative approaches if necessary, but for now, plan for One-Hot Encoding.

  # Analyze 'application_type': Check unique values and if ordinal
  print(df['application_type'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  from sklearn.preprocessing import LabelEncoder

  # Label Encoding for ordinal columns
  label_encode_cols = ['loan_grade', 'loan_subgrade', 'job_experience']
  for col in label_encode_cols:
      le = LabelEncoder()
      df[col] = le.fit_transform(df[col])

  # Identify remaining object columns for One-Hot Encoding
  one_hot_encode_cols = df.select_dtypes(include='object').columns.tolist()

  # One-Hot Encoding for nominal columns
  df = pd.get_dummies(df, columns=one_hot_encode_cols, drop_first=True)
  # Convert boolean columns for state and application_type to float (0.0 or 1.0)
  state_app_cols = [col for col in df.columns if col.startswith('state_code_') or col.startswith('application_type_')]
  for col in state_app_cols:
      df[col] = df[col].astype(float)

  display(df.head())

  # 1. 'ID' column: Unique identifier, not relevant for classification.
  columns_to_drop = ['ID']

  # 2. Highly correlated features:
  # From the correlation matrix, observe highly correlated pairs.
  # 'interest_rate', 'loan_grade', and 'loan_subgrade' are highly correlated.
  # 'total_current_balance' and 'total_revolving_limit' are highly correlated.
  # Choose one from each highly correlated pair to drop.
  # Let's drop 'loan_subgrade' (as loan_grade is a higher level) and 'total_revolving_limit'.
  columns_to_drop.extend(['loan_subgrade', 'total_revolving_limit'])

  # 3. Features with very low absolute correlation with 'default':
  # From the correlation with default output, identify columns with correlation close to 0.
  correlation_matrix = df.corr()

  # Display correlations with the target variable 'default'
  #print('correlation_matrix default value ', correlation_matrix['default'].sort_values(ascending=False))
  low_corr_with_default = correlation_matrix['default'].abs().sort_values()
  # Let's consider dropping features with absolute correlation less than a small threshold, e.g., 0.01
  low_corr_threshold = 0.01
  columns_to_drop.extend(low_corr_with_default[low_corr_with_default < low_corr_threshold].index.tolist())

  # Remove 'default' from the list of columns to drop if it was included due to low correlation
  if 'default' in columns_to_drop:
      columns_to_drop.remove('default')

  # Remove duplicates from the list
  columns_to_drop = list(set(columns_to_drop))

  # 4. Features with potentially low variance:
  # Reviewing describe() output, some one-hot encoded columns might have very low variance if one category is dominant.
  # We can assess variance programmatically if needed, but based on the value counts in the previous step,
  # 'application_type_JOINT' has very few 'JOINT' values, suggesting low variance and potentially low predictive power.
  # 'home_ownership_OTHER' and 'home_ownership_NONE' also have very low counts.
  # Let's add 'application_type_JOINT', 'home_ownership_OTHER', and 'home_ownership_NONE' to the list of columns to drop.
  columns_to_drop.extend(['application_type_JOINT', 'home_ownership_OTHER', 'home_ownership_NONE'])

  # Remove duplicates again after adding more columns
  columns_to_drop = list(set(columns_to_drop))


  print("Columns identified as candidates for dropping:")
  print(columns_to_drop)

  # No code for dropping is needed in this subtask, just identification.




  # Drop the identified unnecessary columns
  # Update columns_to_drop based on the previous analysis and decisions.
  columns_to_drop = [
      'ID',
      'loan_subgrade',
      'total_revolving_limit', # Already handled by not including in log_transform_cols
      'application_type_JOINT',
      'home_ownership_OTHER',
      'home_ownership_NONE',
      # Add columns with very low absolute correlation with 'default' based on the previous output if they are not already in the list
      'state_code_VA', 'state_code_LA', 'state_code_AR', 'state_code_NV', 'total_acc', 'state_code_MO', 'state_code_SC', 'state_code_NJ',
      'state_code_KY', 'state_code_AZ', 'state_code_NH', 'state_code_CO', 'state_code_NY', 'state_code_KS', 'state_code_SD', 'state_code_MN',
      'state_code_FL', 'state_code_OK', 'state_code_GA', 'state_code_MD', 'state_code_ID', 'state_code_NC', 'state_code_OR', 'interest_rate',
      'state_code_VT', 'home_ownership_RENT', 'state_code_MT', 'state_code_MA', 'state_code_TX', 'state_code_NM', 'state_code_CT', 'state_code_WY',
      'state_code_DE', 'state_code_WA', 'state_code_AL', 'state_code_RI', 'state_code_WI', 'loan_purpose_home_improvement', 'state_code_IL',
      'state_code_WV', #'annual_income', # annual_income was log transformed, not dropped
      'state_code_HI'
  ]

  # Remove columns that were log transformed from the drop list
  #columns_to_drop = [col for col in columns_to_drop if col not in log_transform_cols]

  # Remove duplicates
  columns_to_drop = list(set(columns_to_drop))


  df.drop(columns=columns_to_drop, inplace=True)

  # Display the first few rows and the shape of the modified dataframe
  display(df.head())
  print(df.shape)
  return df



In [479]:
df.columns

Index(['ID', 'loan_amnt', 'loan_term', 'interest_rate', 'loan_grade',
       'loan_subgrade', 'job_experience', 'home_ownership', 'annual_income',
       'income_verification_status', 'loan_purpose', 'state_code',
       'debt_to_income', 'delinq_2yrs', 'public_records', 'revolving_balance',
       'total_acc', 'interest_receive', 'application_type', 'last_week_pay',
       'total_current_balance', 'total_revolving_limit', 'default'],
      dtype='object')

In [480]:
df = commonMethodTrain(df)

annual_income            0
delinq_2yrs              0
public_records           0
total_acc                0
last_week_pay            0
total_current_balance    0
total_revolving_limit    0
job_experience           0
dtype: int64
loan_term
3 years    65211
5 years    27963
Name: count, dtype: int64
loan_grade
B    26865
C    25787
A    15534
D    14715
E     7378
F     2344
G      551
Name: count, dtype: int64
loan_subgrade
B4    5879
B3    5879
C2    5479
C1    5443
C3    5270
C4    5182
B2    5169
B5    5095
B1    4843
A5    4723
C5    4413
D1    3716
A4    3631
D2    3239
D3    2759
D4    2717
A3    2450
A1    2377
A2    2353
D5    2284
E1    1924
E2    1736
E3    1513
E4    1228
E5     977
F1     745
F2     545
F3     465
F4     355
F5     234
G1     174
G2     146
G3     105
G5      66
G4      60
Name: count, dtype: int64
job_experience
<5 Years      45312
10+ years     30362
6-10 years    17500
Name: count, dtype: int64
home_ownership
MORTGAGE    46445
RENT        37449
OWN       

,ID,loan_amnt,interest_rate,loan_grade,loan_subgrade,job_experience,annual_income,debt_to_income,delinq_2yrs,public_records,...,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY,application_type_JOINT
0,72199369,9000,9.17,1,6,2,85000.0,26.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14257956,18000,13.65,2,10,2,64000.0,31.67,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66216451,16000,7.26,0,3,2,150000.0,19.70,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46974169,25000,13.99,2,13,2,59800.0,37.39,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,46725961,17000,6.39,0,1,0,72000.0,8.92,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Columns identified as candidates for dropping:
['total_acc', 'home_ownership_RENT', 'state_code_RI', 'state_code_WY', 'loan_subgrade', 'state_code_OR', 'state_code_GA', 'state_code_NY', 'state_code_TX', 'home_ownership_OTHER', 'state_code_MO', 'state_code_NM', 'state_code_NJ', 'state_code_KS', 'state_code_MN', 'state_code_NC', 'state_code_CT', 'state_code_IL', 'application_type_JOINT', 'total_revolving_limit', 'state_code_KY', 'state_code_VT', 'annual_income', 'state_code_DE', 'state_code_NV', 'home_ownership_NONE', 'state_code_FL', 'interest_rate', 'state_code_NH', 'state_code_SC', 'state_code_VA', 'state_code_HI', 'state_code_MT', 'state_code_WA', 'state_code_CO', 'loan_purpose_home_improvement', 'state_code_SD', 'state_code_AL', 'state_code_MD', 'state_code_AZ', 'state_code_MA', 'state_code_LA', 'state_code_OK', 'state_code_WI', 'state_code_ID', 'state_code_WV', 'state_code_AR', 'ID']


,loan_amnt,loan_grade,job_experience,annual_income,debt_to_income,delinq_2yrs,public_records,revolving_balance,interest_receive,last_week_pay,...,state_code_IN,state_code_ME,state_code_MI,state_code_MS,state_code_ND,state_code_NE,state_code_OH,state_code_PA,state_code_TN,state_code_UT
0,9000,1,2,85000.0,26.68,0.0,0.0,39519,59.60,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18000,2,2,64000.0,31.67,0.0,1.0,9783,3348.25,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16000,0,2,150000.0,19.70,2.0,0.0,13641,276.69,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25000,2,2,59800.0,37.39,0.0,0.0,35020,1106.72,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17000,0,0,72000.0,8.92,0.0,0.0,23990,725.29,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(93174, 30)


In [481]:
df.columns

Index(['loan_amnt', 'loan_grade', 'job_experience', 'annual_income',
       'debt_to_income', 'delinq_2yrs', 'public_records', 'revolving_balance',
       'interest_receive', 'last_week_pay', 'total_current_balance', 'default',
       'loan_term_5 years', 'home_ownership_OWN',
       'income_verification_status_Source Verified',
       'income_verification_status_Verified',
       'loan_purpose_debt_consolidation', 'loan_purpose_other',
       'state_code_CA', 'state_code_DC', 'state_code_IN', 'state_code_ME',
       'state_code_MI', 'state_code_MS', 'state_code_ND', 'state_code_NE',
       'state_code_OH', 'state_code_PA', 'state_code_TN', 'state_code_UT'],
      dtype='object')

In [482]:
X = df.drop(['default'],axis=1)
y = df['default']

In [483]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42,stratify=y, shuffle = True)

In [484]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [487]:
sc = StandardScaler()
cols_list = ['loan_amnt',#'total_revolving_limit',
             #'total_acc',
             'annual_income', 'revolving_balance', 'interest_receive', 'last_week_pay', 'total_current_balance', 'debt_to_income']# 'interest_rate']
X_train[cols_list] = sc.fit_transform(X_train[cols_list])
X_test[cols_list] = sc.transform(X_test[cols_list])

In [488]:
def plot(history, name):
    """
    Function to plot loss/accuracy

    history: an object which stores the metrics and losses.
    name: can be one of Loss or Accuracy
    """
    fig, ax = plt.subplots() #Creating a subplot with figure and axes.
    plt.plot(history.history[name]) #Plotting the train accuracy or train loss
    plt.plot(history.history['val_'+name]) #Plotting the validation accuracy or validation loss

    plt.title('Model ' + name.capitalize()) #Defining the title of the plot.
    plt.ylabel(name.capitalize()) #Capitalizing the first letter.
    plt.xlabel('Epoch') #Defining the label for the x-axis.
    fig.legend(['Train', 'Validation'], loc="outside right upper") #Defining the legend, loc controls the position of the legend.

In [489]:
# defining a function to compute different metrics to check performance of a classification model built using statsmodels
def model_performance_classification(
    model, predictors, target, threshold=0.5
):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    threshold: threshold for classifying the observation as class 1
    """

    # checking which probabilities are greater than threshold
    pred = model.predict(predictors) > threshold
    # pred_temp = model.predict(predictors) > threshold
    # # rounding off the above values to get classes
    # pred = np.round(pred_temp)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred, average='weighted')  # to compute Recall
    precision = precision_score(target, pred, average='weighted')  # to compute Precision
    f1 = f1_score(target, pred, average='weighted')  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1 Score": f1,},
        index=[0],
    )

    return df_perf

In [490]:
# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
tf.keras.utils.set_random_seed(812)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()
tf.keras.backend.clear_session()

In [491]:
# Calculate class weights for imbalanced dataset
cw = (y_train.shape[0]) / np.bincount(y_train.astype(int))

# Create a dictionary mapping class indices to their respective class weights
cw_dict = {}
for i in range(cw.shape[0]):
    cw_dict[i] = cw[i]

cw_dict

{0: np.float64(1.31147951204254), 1: np.float64(4.210484032938341)}

In [492]:
def build_model(model, optimizer, predictor_train, target_train, epochs=200, batch_size=128):
  """
  Function to build model and charts as well as to predict

  model : model is a sequential defined model for neural networks
  optimizer: specified optimizer for the model [sgd / adam]
  predictor_train: X_train variable (X_train_smote)
  target_train: y_train variable (y_train_smote)
  """

  # clears the current Keras session, resetting all layers and models previously created, freeing up memory and resources.
  tf.keras.backend.clear_session()
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Setting binary entropy as the loss function

  early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=35,          # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True # Restore the best weights found during training
  )

  start = time.time()
  history = model.fit(
    predictor_train, target_train,
    batch_size=batch_size,
    validation_data=(X_test,y_test),
    epochs=epochs,
    verbose=0,
    class_weight=cw_dict,
    callbacks=[early_stopping]
  )
  end=time.time()
  print('Total time taken for the model to run  ==> ', end-start , " secs")
  #plot(history,'loss') # Plotting loss graph
  #plot(history,'recall') # Plotting recall graph
  #Predicting the results using best as a threshold
  y_train_pred = model.predict(predictor_train)
  y_train_pred = (y_train_pred > 0.5)
  print('y_train_pred', y_train_pred)
  #Predicting the results using best as a threshold
  y_test_pred = model.predict(X_test)
  y_test_pred = (y_test_pred > 0.5)
  print('y_test_pred', y_test_pred)
  print('y_test', y_test)
  return history

In [493]:
#Initializing the neural network
%%time
model_3 = Sequential()
model_3.add(Dense(256,activation="relu",input_dim=X_train.shape[1]))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.3))
model_3.add(Dense(128,activation="relu", kernel_initializer="he_normal"))
model_3.add(Dense(64,activation="relu", kernel_initializer="he_normal"))
model_3.add(Dropout(0.2))
model_3.add(Dense(32,activation="tanh"))
model_3.add(Dense(16,activation="relu"))
model_3.add(Dropout(0.1))
model_3.add(Dense(8,activation="tanh"))
model_3.add(Dense(1,activation="sigmoid"))
optimizer_3 = tf.keras.optimizers.Adam(0.0001)    # defining Adam as the optimizer to be used
epochs = 1024
batch_size = 256
build_model(model_3, optimizer_3, X_train, y_train, epochs, batch_size)
model_3_train_perf_name = model_performance_classification(model_3, X_train, y_train)
model_3_train_perf_name

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         7,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,609 (205.50 KB)

 Trainable params: 52,097 (203.50 KB)

 Non-trainable params: 512 (2.00 KB)

Total time taken for the model to run  ==>  342.48454785346985  secs
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
y_train_pred [[ True]
 [False]
 [False]
 ...
 [False]
 [ True]
 [False]]
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
y_test_pred [[False]
 [False]
 [ True]
 ...
 [ True]
 [False]
 [ True]]
y_test [1 0 0 ... 0 0 0]
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
CPU times: user 7min 59s, sys: 45.9 s, total: 8min 45s
Wall time: 5min 53s


,Accuracy,Recall,Precision,F1 Score
0,0.749714,0.749714,0.801374,0.764506


In [494]:
model_3_test_perf_name = model_performance_classification(model_3, X_test, y_test)
model_3_test_perf_name

292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,Accuracy,Recall,Precision,F1 Score
0,0.720541,0.720541,0.774357,0.737055


#TEST

In [495]:
test_data = pd.read_csv("/content/drive/MyDrive/AIML_UT/Colab_Files/Hackathons/UT-Hackathon-2/Test_set.csv")
test_df = test_data.copy()

In [496]:
def commonMethodTrainA(df):
  # Calculate the mode of 'job_experience' and fill missing values
  job_experience_mode = df['job_experience'].mode()[0]
  df['job_experience'].fillna(job_experience_mode, inplace=True)

  # Calculate the median for the specified numerical columns and fill missing values
  numerical_cols_to_impute = ['annual_income', 'delinq_2yrs', 'public_records', 'total_acc', 'last_week_pay', 'total_current_balance', 'total_revolving_limit']
  for col in numerical_cols_to_impute:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

  # Verify that there are no more missing values in the imputed columns
  print(df[numerical_cols_to_impute + ['job_experience']].isnull().sum())


  # Analyze each categorical column and determine the encoding strategy

  # Columns identified as object type:
  # ['loan_term', 'loan_grade', 'loan_subgrade', 'job_experience', 'home_ownership', 'income_verification_status', 'loan_purpose', 'state_code', 'application_type']

  # Analyze 'loan_term': Check unique values and if ordinal
  print(df['loan_term'].value_counts())
  # This is likely nominal or ordinal (shorter term vs longer term). Given the limited unique values, One-Hot Encoding is suitable.

  # Analyze 'loan_grade': Check unique values and if ordinal
  print(df['loan_grade'].value_counts())
  # This is ordinal (A is better than B, etc.). Label Encoding is suitable.

  # Analyze 'loan_subgrade': Check unique values and if ordinal
  print(df['loan_subgrade'].value_counts())
  # This is ordinal (A1 is better than A2, etc.). Label Encoding is suitable, but it has many unique values, so One-Hot Encoding might also be considered depending on the model. Let's start with Label Encoding and re-evaluate if necessary.

  # Analyze 'job_experience': Check unique values and if ordinal
  print(df['job_experience'].value_counts())
  # This is ordinal (e.g., 10+ years is more than <5 years). Label Encoding is suitable.

  # Analyze 'home_ownership': Check unique values and if ordinal
  print(df['home_ownership'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'income_verification_status': Check unique values and if ordinal
  print(df['income_verification_status'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'loan_purpose': Check unique values and if ordinal
  print(df['loan_purpose'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  # Analyze 'state_code': Check unique values and if ordinal
  print(df['state_code'].value_counts())
  # This is nominal and has many unique values. One-Hot Encoding will result in a very wide matrix. Consider alternative approaches if necessary, but for now, plan for One-Hot Encoding.

  # Analyze 'application_type': Check unique values and if ordinal
  print(df['application_type'].value_counts())
  # This is nominal. One-Hot Encoding is suitable.

  from sklearn.preprocessing import LabelEncoder

  # Label Encoding for ordinal columns
  label_encode_cols = ['loan_grade', 'loan_subgrade', 'job_experience']
  for col in label_encode_cols:
      le = LabelEncoder()
      df[col] = le.fit_transform(df[col])

  # Identify remaining object columns for One-Hot Encoding
  one_hot_encode_cols = df.select_dtypes(include='object').columns.tolist()

  # One-Hot Encoding for nominal columns
  df = pd.get_dummies(df, columns=one_hot_encode_cols, drop_first=True)
  # Convert boolean columns for state and application_type to float (0.0 or 1.0)
  state_app_cols = [col for col in df.columns if col.startswith('state_code_') or col.startswith('application_type_')]
  for col in state_app_cols:
      df[col] = df[col].astype(float)

  display(df.head())

  # 1. 'ID' column: Unique identifier, not relevant for classification.
  columns_to_drop = ['ID']

  # 2. Highly correlated features:
  # From the correlation matrix, observe highly correlated pairs.
  # 'interest_rate', 'loan_grade', and 'loan_subgrade' are highly correlated.
  # 'total_current_balance' and 'total_revolving_limit' are highly correlated.
  # Choose one from each highly correlated pair to drop.
  # Let's drop 'loan_subgrade' (as loan_grade is a higher level) and 'total_revolving_limit'.
  columns_to_drop.extend(['loan_subgrade', 'total_revolving_limit'])

  # 3. Features with very low absolute correlation with 'default':
  # From the correlation with default output, identify columns with correlation close to 0.
  correlation_matrix = df.corr()

  # Display correlations with the target variable 'default'
  #print('correlation_matrix default value ', correlation_matrix['default'].sort_values(ascending=False))
  #low_corr_with_default = correlation_matrix['default'].abs().sort_values()
  # Let's consider dropping features with absolute correlation less than a small threshold, e.g., 0.01
  low_corr_threshold = 0.01
  #columns_to_drop.extend(low_corr_with_default[low_corr_with_default < low_corr_threshold].index.tolist())

  # Remove 'default' from the list of columns to drop if it was included due to low correlation
  if 'default' in columns_to_drop:
      columns_to_drop.remove('default')

  # Remove duplicates from the list
  columns_to_drop = list(set(columns_to_drop))

  # 4. Features with potentially low variance:
  # Reviewing describe() output, some one-hot encoded columns might have very low variance if one category is dominant.
  # We can assess variance programmatically if needed, but based on the value counts in the previous step,
  # 'application_type_JOINT' has very few 'JOINT' values, suggesting low variance and potentially low predictive power.
  # 'home_ownership_OTHER' and 'home_ownership_NONE' also have very low counts.
  # Let's add 'application_type_JOINT', 'home_ownership_OTHER', and 'home_ownership_NONE' to the list of columns to drop.
  columns_to_drop.extend(['application_type_JOINT', 'home_ownership_OTHER', 'home_ownership_NONE'])

  # Remove duplicates again after adding more columns
  columns_to_drop = list(set(columns_to_drop))


  print("Columns identified as candidates for dropping:")
  print(columns_to_drop)

  # No code for dropping is needed in this subtask, just identification.




  # Drop the identified unnecessary columns
  # Update columns_to_drop based on the previous analysis and decisions.
  columns_to_drop = [
      'ID',
      'loan_subgrade',
      'total_revolving_limit', # Already handled by not including in log_transform_cols
      'application_type_JOINT',
      'home_ownership_OTHER',
      'home_ownership_NONE',
      # Add columns with very low absolute correlation with 'default' based on the previous output if they are not already in the list
      'state_code_VA', 'state_code_LA', 'state_code_AR', 'state_code_NV', 'total_acc', 'state_code_MO', 'state_code_SC', 'state_code_NJ',
      'state_code_KY', 'state_code_AZ', 'state_code_NH', 'state_code_CO', 'state_code_NY', 'state_code_KS', 'state_code_SD', 'state_code_MN',
      'state_code_FL', 'state_code_OK', 'state_code_GA', 'state_code_MD', 'state_code_ID', 'state_code_NC', 'state_code_OR', 'interest_rate',
      'state_code_VT', 'home_ownership_RENT', 'state_code_MT', 'state_code_MA', 'state_code_TX', 'state_code_NM', 'state_code_CT', 'state_code_WY',
      'state_code_DE', 'state_code_WA', 'state_code_AL', 'state_code_RI', 'state_code_WI', 'loan_purpose_home_improvement', 'state_code_IL',
      'state_code_WV', #'annual_income', # annual_income was log transformed, not dropped
      'state_code_HI'
  ]

  # Remove columns that were log transformed from the drop list
  #columns_to_drop = [col for col in columns_to_drop if col not in log_transform_cols]

  # Remove duplicates
  columns_to_drop = list(set(columns_to_drop))


  df.drop(columns=columns_to_drop, inplace=True)

  # Display the first few rows and the shape of the modified dataframe
  display(df.head())
  print(df.shape)
  return df



In [497]:
test_df = commonMethodTrainA(test_df)

annual_income            0
delinq_2yrs              0
public_records           0
total_acc                0
last_week_pay            0
total_current_balance    0
total_revolving_limit    0
job_experience           0
dtype: int64
loan_term
3 years    28110
5 years    11823
Name: count, dtype: int64
loan_grade
B    11551
C    10948
A     6764
D     6237
E     3158
F     1030
G      245
Name: count, dtype: int64
loan_subgrade
B3    2547
B4    2519
C1    2436
C2    2371
B5    2247
C3    2220
B2    2206
C4    2084
A5    2069
B1    2032
C5    1837
D1    1576
A4    1559
D2    1373
D3    1175
D4    1141
A3    1074
A1    1067
A2     995
D5     972
E1     803
E2     755
E3     652
E4     519
E5     429
F1     312
F2     254
F3     195
F4     147
F5     122
G2      72
G1      71
G3      46
G5      31
G4      25
Name: count, dtype: int64
job_experience
<5 Years      19379
10+ years     13146
6-10 years     7408
Name: count, dtype: int64
home_ownership
MORTGAGE    20008
RENT        15938
OWN       

,ID,loan_amnt,interest_rate,loan_grade,loan_subgrade,job_experience,annual_income,debt_to_income,delinq_2yrs,public_records,...,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY,application_type_JOINT
0,4855329,12000,15.31,2,11,2,73400.0,14.62,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,66862420,12000,7.26,0,3,0,105000.0,11.38,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3637416,15000,14.33,2,10,1,50000.0,28.15,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53682249,12000,9.99,1,7,1,37000.0,34.32,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,53937165,20150,11.53,1,9,2,75000.0,26.74,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Columns identified as candidates for dropping:
['loan_subgrade', 'application_type_JOINT', 'total_revolving_limit', 'home_ownership_OTHER', 'home_ownership_NONE', 'ID']


,loan_amnt,loan_grade,job_experience,annual_income,debt_to_income,delinq_2yrs,public_records,revolving_balance,interest_receive,last_week_pay,...,state_code_IN,state_code_ME,state_code_MI,state_code_MS,state_code_ND,state_code_NE,state_code_OH,state_code_PA,state_code_TN,state_code_UT
0,12000,2,2,73400.0,14.62,0.0,0.0,22156,2290.24,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12000,0,0,105000.0,11.38,0.0,0.0,7592,202.68,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15000,2,1,50000.0,28.15,0.0,1.0,17983,1166.24,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12000,1,1,37000.0,34.32,0.0,0.0,12262,635.06,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20150,1,2,75000.0,26.74,1.0,0.0,8251,1232.84,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(39933, 29)


In [498]:
df.columns

Index(['loan_amnt', 'loan_grade', 'job_experience', 'annual_income',
       'debt_to_income', 'delinq_2yrs', 'public_records', 'revolving_balance',
       'interest_receive', 'last_week_pay', 'total_current_balance', 'default',
       'loan_term_5 years', 'home_ownership_OWN',
       'income_verification_status_Source Verified',
       'income_verification_status_Verified',
       'loan_purpose_debt_consolidation', 'loan_purpose_other',
       'state_code_CA', 'state_code_DC', 'state_code_IN', 'state_code_ME',
       'state_code_MI', 'state_code_MS', 'state_code_ND', 'state_code_NE',
       'state_code_OH', 'state_code_PA', 'state_code_TN', 'state_code_UT'],
      dtype='object')

In [499]:
sc = StandardScaler()
cols_list = ['loan_amnt',#'total_revolving_limit',
             #'total_acc',
             'annual_income', 'revolving_balance', 'interest_receive', 'last_week_pay', 'total_current_balance', 'debt_to_income']#, 'interest_rate']
test_df[cols_list] = sc.fit_transform(test_df[cols_list])

In [500]:
# Make predictions on the test data (P) using model_3
test_predictions = model_3.predict(test_df)
test_predictions

1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


array([[0.67094266],
       [0.30814195],
       [0.5075776 ],
       ...,
       [0.0360349 ],
       [0.14203632],
       [0.2557951 ]], dtype=float32)

In [501]:
# The previous error occurred because you were passing the continuous predictions as the target.
# To get the binary predictions from the model, you can apply a threshold (e.g., 0.5).
binary_test_predictions = (test_predictions > 0.4).astype(int)
print("Binary Test Predictions:")
print(binary_test_predictions)

# To evaluate the model performance on the test set, you would need the true binary labels for the test data.
# If you had the true test labels (e.g., in a variable called y_test_true), you would use:
# model_3_final_test = model_performance_classification(model_3, P, y_test_true)
# model_3_final_test

Binary Test Predictions:
[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]


Let's select a specific ID from the original `test_data` to predict.

In [502]:
# Assuming you want to predict for the first ID in the original test data for demonstration
# You can change the index [0] to select a different row
id_to_predict = test_data.loc[0, 'ID']
print(f"Selected ID for prediction: {id_to_predict}")

# Get the row from the preprocessed test_df corresponding to the selected ID
# Note: We need to use the preprocessed test_df for prediction
row_for_prediction = test_df[test_data['ID'] == id_to_predict]

# Ensure the row has the same columns and order as the training data (X_train)
# This is important for the model to make a correct prediction
# You might need to add missing columns filled with 0 if your test set preprocessing results in fewer columns
# compared to the training set preprocessing due to different unique values in categorical features.
# A more robust approach would involve saving the column order from the training data preprocessing.

# For simplicity in this example, we assume the columns match.
# If you encounter errors, you might need to realign the columns:
# row_for_prediction = row_for_prediction.reindex(columns=X_train.columns, fill_value=0)


# Make the prediction using the trained model
prediction_for_id = model_3.predict(row_for_prediction)

# Apply the threshold to get the binary prediction
binary_prediction_for_id = (prediction_for_id > 0.5).astype(int)

print(f"Prediction for ID {id_to_predict}: {binary_prediction_for_id[0][0]}")

if binary_prediction_for_id[0][0] == 1:
    print(f"Based on the model, ID {id_to_predict} is predicted to default.")
else:
    print(f"Based on the model, ID {id_to_predict} is predicted not to default.")

Selected ID for prediction: 4855329
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction for ID 4855329: 1
Based on the model, ID 4855329 is predicted to default.


In [503]:
# Make predictions on the entire preprocessed test DataFrame (test_df)
all_test_predictions = model_3.predict(test_df)

# Apply the threshold to get binary predictions (0 or 1)
binary_all_test_predictions = (all_test_predictions > 0.4).astype(int)

# Get the original 'ID' column from the test_data DataFrame
# Ensure that the order of IDs in test_data matches the order of rows in test_df after preprocessing
# (which should be the case if preprocessing didn't reorder rows based on index values)
original_ids = test_data['ID']

# Create a new DataFrame with 'ID' and 'default' columns
# The 'default' column will contain the binary predictions
output_df = pd.DataFrame({'ID': original_ids, 'default': binary_all_test_predictions.flatten()})

# Define the output file path in the Colab environment
output_filename = 'test_predictions10.csv'
output_path = output_filename

# Save the DataFrame to a CSV file in the Colab environment
output_df.to_csv(output_path, index=False)



# Download the file to your local machine
from google.colab import files
files.download(output_path)
test_predictions

1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

array([[0.67094266],
       [0.30814195],
       [0.5075776 ],
       ...,
       [0.0360349 ],
       [0.14203632],
       [0.2557951 ]], dtype=float32)

In [504]:
print(f"Predictions for all IDs saved to: {output_path}")
display(output_df.head())

Predictions for all IDs saved to: test_predictions10.csv


,ID,default
0,4855329,1
1,66862420,0
2,3637416,1
3,53682249,0
4,53937165,0


# Task
Improve the accuracy of the existing neural network model for predicting loan defaults.

## Review model architecture

### Subtask:
Examine the current neural network architecture (number of layers, neurons per layer, activation functions) to identify potential areas for modification.


**Reasoning**:
Analyze the existing neural network architecture based on the `model_3.summary()` output to identify potential improvements.



In [87]:
print("Current Model Architecture:")
model_3.summary()

print("\nAnalysis of the Architecture:")
print("- Input Layer: The first Dense layer has an input dimension matching the number of features in the training data.")
print("- Hidden Layers: The model uses multiple Dense layers with varying numbers of neurons (128, 64, 32, 16, 8).")
print("- Activation Functions: Sigmoid is used in the first Dense layer, ReLU in the second, third, and fifth Dense layers, and Tanh in the fourth Dense layer. Sigmoid is used in the output layer.")
print("- Regularization/Normalization: BatchNormalization is applied after the first Dense layer, and Dropout layers are used after the BatchNormalization layer and after the third and fifth Dense layers with rates 0.4, 0.3, and 0.2 respectively.")
print("- Output Layer: A single neuron Dense layer with a sigmoid activation is used, which is appropriate for binary classification.")

print("\nPotential Areas for Modification:")
print("- Activation Functions: Using ReLU in the first hidden layer might be more common than Sigmoid. The mix of ReLU and Tanh in subsequent layers could be simplified. Consider using ReLU for all hidden layers.")
print("- Number of Neurons: The number of neurons decreases significantly in later layers. Experimenting with different configurations or a more gradual decrease might be beneficial.")
print("- Dropout Rates: The dropout rates seem reasonable, but fine-tuning them could help manage overfitting.")
print("- Batch Normalization Placement: Batch Normalization is after the first dense layer. Exploring placing it before the activation or in other layers could be considered.")
print("- Number of Layers: The current depth of the network (5 hidden layers) might be sufficient, but experimenting with more or fewer layers could be explored.")

Current Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,573 (178.02 KB)

 Trainable params: 15,105 (59.00 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 30,212 (118.02 KB)


Analysis of the Architecture:
- Input Layer: The first Dense layer has an input dimension matching the number of features in the training data.
- Hidden Layers: The model uses multiple Dense layers with varying numbers of neurons (128, 64, 32, 16, 8).
- Activation Functions: Sigmoid is used in the first Dense layer, ReLU in the second, third, and fifth Dense layers, and Tanh in the fourth Dense layer. Sigmoid is used in the output layer.
- Regularization/Normalization: BatchNormalization is applied after the first Dense layer, and Dropout layers are used after the BatchNormalization layer and after the third and fifth Dense layers with rates 0.4, 0.3, and 0.2 respectively.
- Output Layer: A single neuron Dense layer with a sigmoid activation is used, which is appropriate for binary classification.

Potential Areas for Modification:
- Activation Functions: Using ReLU in the first hidden layer might be more common than Sigmoid. The mix of ReLU and Tanh in subsequent layers could be simp

## Hyperparameter tuning

### Subtask:
Experiment with different hyperparameters for the current neural network, such as learning rate for the optimizer, dropout rates, and L2 regularization.


**Reasoning**:
Define a function to build the Keras model with hyperparameters as arguments and then use KerasClassifier and RandomizedSearchCV to find the best hyperparameters.



In [88]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_keras_model(learning_rate=0.001, dropout_rate_1=0.4, dropout_rate_2=0.3, dropout_rate_3=0.2, l2_reg=0.0):
    """
    Builds a Keras Sequential model with configurable hyperparameters.

    Args:
        learning_rate (float): Learning rate for the Adam optimizer.
        dropout_rate_1 (float): Dropout rate after the first hidden layer.
        dropout_rate_2 (float): Dropout rate after the third hidden layer.
        dropout_rate_3 (float): Dropout rate after the fifth hidden layer.
        l2_reg (float): L2 regularization strength for dense layers.

    Returns:
        keras.Sequential: The built Keras model.
    """
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(128, activation="sigmoid", input_dim=X_train.shape[1], kernel_regularizer=L2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(32, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(16, activation="tanh", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(8, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper
keras_clf = KerasClassifier(model=build_keras_model, epochs=10, batch_size=128, verbose=0, class_weight=cw_dict)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'model__learning_rate': [0.0005, 0.001, 0.005, 0.01],
    'model__dropout_rate_1': [0.3, 0.4, 0.5],
    'model__dropout_rate_2': [0.2, 0.3, 0.4],
    'model__dropout_rate_3': [0.1, 0.2, 0.3],
    'model__l2_reg': [0.0, 0.001, 0.01]
}

# Set up RandomizedSearchCV
# Using a smaller number of iterations (n_iter) and fewer cross-validation folds (cv) for faster execution during experimentation.
# In a real scenario, you would increase n_iter and cv.
random_search = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42)

# Perform the random search
print("Starting RandomizedSearchCV...")
random_search_result = random_search.fit(X_train, y_train)
print("RandomizedSearchCV finished.")

# Print the best hyperparameters and the corresponding score
print("Best Score: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Store the best model found by RandomizedSearchCV
best_keras_model = random_search_result.best_estimator_

ModuleNotFoundError: No module named 'scikeras'

**Reasoning**:
The previous command failed because the `scikeras` library is not installed. Install the required library.



In [89]:
!pip install scikeras

**Reasoning**:
Now that scikeras is installed, attempt to run the code to define the model building function and perform hyperparameter tuning using RandomizedSearchCV.



In [90]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_keras_model(learning_rate=0.001, dropout_rate_1=0.4, dropout_rate_2=0.3, dropout_rate_3=0.2, l2_reg=0.0):
    """
    Builds a Keras Sequential model with configurable hyperparameters.

    Args:
        learning_rate (float): Learning rate for the Adam optimizer.
        dropout_rate_1 (float): Dropout rate after the first hidden layer.
        dropout_rate_2 (float): Dropout rate after the third hidden layer.
        dropout_rate_3 (float): Dropout rate after the fifth hidden layer.
        l2_reg (float): L2 regularization strength for dense layers.

    Returns:
        keras.Sequential: The built Keras model.
    """
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(128, activation="sigmoid", input_dim=X_train.shape[1], kernel_regularizer=L2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(32, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(16, activation="tanh", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(8, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper
keras_clf = KerasClassifier(model=build_keras_model, epochs=10, batch_size=128, verbose=0, class_weight=cw_dict)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'model__learning_rate': [0.0005, 0.001, 0.005, 0.01],
    'model__dropout_rate_1': [0.3, 0.4, 0.5],
    'model__dropout_rate_2': [0.2, 0.3, 0.4],
    'model__dropout_rate_3': [0.1, 0.2, 0.3],
    'model__l2_reg': [0.0, 0.001, 0.01]
}

# Set up RandomizedSearchCV
# Using a smaller number of iterations (n_iter) and fewer cross-validation folds (cv) for faster execution during experimentation.
# In a real scenario, you would increase n_iter and cv.
random_search = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42)

# Perform the random search
print("Starting RandomizedSearchCV...")
random_search_result = random_search.fit(X_train, y_train)
print("RandomizedSearchCV finished.")

# Print the best hyperparameters and the corresponding score
print("Best Score: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Store the best model found by RandomizedSearchCV
best_keras_model = random_search_result.best_estimator_

Starting RandomizedSearchCV...


AttributeError: 'super' object has no attribute '__sklearn_tags__'

**Reasoning**:
The error "AttributeError: 'super' object has no attribute '__sklearn_tags__'" suggests an incompatibility between the scikit-learn version and scikeras version or an issue with how the KerasClassifier is being used within the scikit-learn framework. Let's try updating scikit-learn to a version compatible with the installed scikeras.



In [91]:
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 131.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


**Reasoning**:
Now that scikit-learn has been updated, attempt to run the code again to define the model building function and perform hyperparameter tuning using RandomizedSearchCV.



In [92]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_keras_model(learning_rate=0.001, dropout_rate_1=0.4, dropout_rate_2=0.3, dropout_rate_3=0.2, l2_reg=0.0):
    """
    Builds a Keras Sequential model with configurable hyperparameters.

    Args:
        learning_rate (float): Learning rate for the Adam optimizer.
        dropout_rate_1 (float): Dropout rate after the first hidden layer.
        dropout_rate_2 (float): Dropout rate after the third hidden layer.
        dropout_rate_3 (float): Dropout rate after the fifth hidden layer.
        l2_reg (float): L2 regularization strength for dense layers.

    Returns:
        keras.Sequential: The built Keras model.
    """
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(128, activation="sigmoid", input_dim=X_train.shape[1], kernel_regularizer=L2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(32, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(16, activation="tanh", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(8, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper
keras_clf = KerasClassifier(model=build_keras_model, epochs=10, batch_size=128, verbose=0, class_weight=cw_dict)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'model__learning_rate': [0.0005, 0.001, 0.005, 0.01],
    'model__dropout_rate_1': [0.3, 0.4, 0.5],
    'model__dropout_rate_2': [0.2, 0.3, 0.4],
    'model__dropout_rate_3': [0.1, 0.2, 0.3],
    'model__l2_reg': [0.0, 0.001, 0.01]
}

# Set up RandomizedSearchCV
# Using a smaller number of iterations (n_iter) and fewer cross-validation folds (cv) for faster execution during experimentation.
# In a real scenario, you would increase n_iter and cv.
random_search = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42)

# Perform the random search
print("Starting RandomizedSearchCV...")
random_search_result = random_search.fit(X_train, y_train)
print("RandomizedSearchCV finished.")

# Print the best hyperparameters and the corresponding score
print("Best Score: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Store the best model found by RandomizedSearchCV
best_keras_model = random_search_result.best_estimator_

Starting RandomizedSearchCV...


AttributeError: 'super' object has no attribute '__sklearn_tags__'

## Hyperparameter tuning retry 1

### Subtask:
Retry hyperparameter tuning for the neural network model using RandomizedSearchCV, addressing the previous `AttributeError` by ensuring compatibility of libraries or exploring alternative tuning approaches if the error persists.


**Reasoning**:
Retry hyperparameter tuning using RandomizedSearchCV, ensuring scikeras is installed and compatible, and addressing the Attribute Error if it persists.



In [93]:
# Re-install scikeras and ensure compatibility (already done in previous steps, but including for completeness)
# !pip install scikeras -U
# !pip install -U scikit-learn

# Re-define the build_keras_model function and the KerasClassifier wrapper
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_keras_model(learning_rate=0.001, dropout_rate_1=0.4, dropout_rate_2=0.3, dropout_rate_3=0.2, l2_reg=0.0):
    """
    Builds a Keras Sequential model with configurable hyperparameters.

    Args:
        learning_rate (float): Learning rate for the Adam optimizer.
        dropout_rate_1 (float): Dropout rate after the first hidden layer.
        dropout_rate_2 (float): Dropout rate after the third hidden layer.
        dropout_rate_3 (float): Dropout rate after the fifth hidden layer.
        l2_reg (float): L2 regularization strength for dense layers.

    Returns:
        keras.Sequential: The built Keras model.
    """
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(128, activation="sigmoid", input_dim=X_train.shape[1], kernel_regularizer=L2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(32, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(16, activation="tanh", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(8, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper
# Set epochs to a lower number for faster tuning, increase for final model
keras_clf = KerasClassifier(model=build_keras_model, epochs=20, batch_size=256, verbose=0, class_weight=cw_dict)

# Re-define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'model__learning_rate': [0.0005, 0.001, 0.005],
    'model__dropout_rate_1': [0.3, 0.4, 0.5],
    'model__dropout_rate_2': [0.2, 0.3, 0.4],
    'model__dropout_rate_3': [0.1, 0.2, 0.3],
    'model__l2_reg': [0.0, 0.001, 0.01]
}

# Set up RandomizedSearchCV
# Using a smaller number of iterations (n_iter) and fewer cross-validation folds (cv) for faster execution during experimentation.
# In a real scenario, you would increase n_iter and cv.
random_search = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_dist, n_iter=5, cv=2, scoring='accuracy', random_state=42)

# Perform the random search
print("Starting RandomizedSearchCV...")
try:
    random_search_result = random_search.fit(X_train, y_train)
    print("RandomizedSearchCV finished.")

    # Print the best hyperparameters and the corresponding score
    print("Best Score: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

    # Store the best model found by RandomizedSearchCV
    best_keras_model = random_search_result.best_estimator_

except AttributeError as e:
    print(f"AttributeError during RandomizedSearchCV: {e}")
    print("Attempting manual tuning as an alternative approach.")
    # If the Attribute Error persists, attempt manual tuning or note that an alternative is needed.
    # For manual tuning, we would train models with specific parameter combinations here
    # and evaluate them. Since the instructions allow for failure if workarounds are needed,
    # and a full manual tuning implementation is complex and goes beyond a simple retry,
    # we will note the need for an alternative and finish the task.
    best_keras_model = None # Indicate that tuning failed

# Note: If manual tuning were to be fully implemented, you would iterate through
# selected parameter combinations, train a model for each, evaluate its performance,
# and keep track of the best performing model and its parameters.


Starting RandomizedSearchCV...
AttributeError during RandomizedSearchCV: 'super' object has no attribute '__sklearn_tags__'
Attempting manual tuning as an alternative approach.


## Summary:

### Data Analysis Key Findings

*   The initial neural network model architecture consists of multiple dense layers with a mix of activation functions (Sigmoid, ReLU, Tanh), batch normalization after the first dense layer, and dropout layers with varying rates.
*   Potential areas for improving the model architecture include standardizing activation functions (potentially using ReLU), experimenting with the number of neurons and their distribution across layers, fine-tuning dropout rates, exploring batch normalization placement, and testing different numbers of hidden layers.
*   Attempts to perform automated hyperparameter tuning using `RandomizedSearchCV` with `scikeras` failed due to a persistent `AttributeError`, indicating a compatibility issue between the libraries.
*   Despite retrying and attempting to address the compatibility issue, the `RandomizedSearchCV` process could not be completed, and hyperparameter tuning was unsuccessful within the scope of the provided steps.

### Insights or Next Steps

*   Investigate and resolve the compatibility issue between `scikeras` and `scikit-learn` to enable the use of `RandomizedSearchCV` or explore alternative hyperparameter tuning frameworks like Keras Tuner.
*   Based on the architectural analysis, manually experiment with variations in activation functions (e.g., all ReLU in hidden layers), neuron counts, and the placement of batch normalization to assess their impact on model performance if automated tuning remains blocked.


## Review model architecture

### Subtask:
Examine the current neural network architecture (number of layers, neurons per layer, activation functions) to identify potential areas for modification.

**Reasoning**:
Analyze the existing neural network architecture based on the `model_3.summary()` output to identify potential improvements.

In [94]:
print("Current Model Architecture:")
model_3.summary()

print("\nAnalysis of the Architecture:")
print("- Input Layer: The first Dense layer has an input dimension matching the number of features in the training data.")
print("- Hidden Layers: The model uses multiple Dense layers with varying numbers of neurons (128, 64, 32, 16, 8).")
print("- Activation Functions: Sigmoid is used in the first Dense layer, ReLU in the second, third, and fifth Dense layers, and Tanh in the fourth Dense layer. Sigmoid is used in the output layer.")
print("- Regularization/Normalization: BatchNormalization is applied after the first Dense layer, and Dropout layers are used after the BatchNormalization layer and after the third and fifth Dense layers with rates 0.4, 0.3, and 0.2 respectively.")
print("- Output Layer: A single neuron Dense layer with a sigmoid activation is used, which is appropriate for binary classification.")

print("\nPotential Areas for Modification:")
print("- Activation Functions: Using ReLU in the first hidden layer might be more common than Sigmoid. The mix of ReLU and Tanh in subsequent layers could be simplified. Consider using ReLU for all hidden layers.")
print("- Number of Neurons: The number of neurons decreases significantly in later layers. Experimenting with different configurations or a more gradual decrease might be beneficial.")
print("- Dropout Rates: The dropout rates seem reasonable, but fine-tuning them could help manage overfitting.")
print("- Batch Normalization Placement: Batch Normalization is after the first dense layer. Exploring placing it before the activation or in other layers could be considered.")
print("- Number of Layers: The current depth of the network (5 hidden layers) might be sufficient, but experimenting with more or fewer layers could be explored.")

Current Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,573 (178.02 KB)

 Trainable params: 15,105 (59.00 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 30,212 (118.02 KB)


Analysis of the Architecture:
- Input Layer: The first Dense layer has an input dimension matching the number of features in the training data.
- Hidden Layers: The model uses multiple Dense layers with varying numbers of neurons (128, 64, 32, 16, 8).
- Activation Functions: Sigmoid is used in the first Dense layer, ReLU in the second, third, and fifth Dense layers, and Tanh in the fourth Dense layer. Sigmoid is used in the output layer.
- Regularization/Normalization: BatchNormalization is applied after the first Dense layer, and Dropout layers are used after the BatchNormalization layer and after the third and fifth Dense layers with rates 0.4, 0.3, and 0.2 respectively.
- Output Layer: A single neuron Dense layer with a sigmoid activation is used, which is appropriate for binary classification.

Potential Areas for Modification:
- Activation Functions: Using ReLU in the first hidden layer might be more common than Sigmoid. The mix of ReLU and Tanh in subsequent layers could be simp

## Hyperparameter tuning

### Subtask:
Experiment with different hyperparameters for the current neural network, such as learning rate for the optimizer, dropout rates, and L2 regularization.

**Reasoning**:
Define a function to build the Keras model with hyperparameters as arguments and then use KerasClassifier and RandomizedSearchCV to find the best hyperparameters.

In [95]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_keras_model(learning_rate=0.001, dropout_rate_1=0.4, dropout_rate_2=0.3, dropout_rate_3=0.2, l2_reg=0.0):
    """
    Builds a Keras Sequential model with configurable hyperparameters.

    Args:
        learning_rate (float): Learning rate for the Adam optimizer.
        dropout_rate_1 (float): Dropout rate after the first hidden layer.
        dropout_rate_2 (float): Dropout rate after the third hidden layer.
        dropout_rate_3 (float): Dropout rate after the fifth hidden layer.
        l2_reg (float): L2 regularization strength for dense layers.

    Returns:
        keras.Sequential: The built Keras model.
    """
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(128, activation="sigmoid", input_dim=X_train.shape[1], kernel_regularizer=L2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(32, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(16, activation="tanh", kernel_regularizer=L2(l2_reg)))
    model.add(Dense(8, activation="relu", kernel_regularizer=L2(l2_reg)))
    model.add(Dropout(dropout_rate_3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper
keras_clf = KerasClassifier(model=build_keras_model, epochs=10, batch_size=128, verbose=0, class_weight=cw_dict)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'model__learning_rate': [0.0005, 0.001, 0.005, 0.01],
    'model__dropout_rate_1': [0.3, 0.4, 0.5],
    'model__dropout_rate_2': [0.2, 0.3, 0.4],
    'model__dropout_rate_3': [0.1, 0.2, 0.3],
    'model__l2_reg': [0.0, 0.001, 0.01]
}

# Set up RandomizedSearchCV
# Using a smaller number of iterations (n_iter) and fewer cross-validation folds (cv) for faster execution during experimentation.
# In a real scenario, you would increase n_iter and cv.
random_search = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42)

# Perform the random search
print("Starting RandomizedSearchCV...")
random_search_result = random_search.fit(X_train, y_train)
print("RandomizedSearchCV finished.")

# Print the best hyperparameters and the corresponding score
print("Best Score: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Store the best model found by RandomizedSearchCV
best_keras_model = random_search_result.best_estimator_

Starting RandomizedSearchCV...


AttributeError: 'super' object has no attribute '__sklearn_tags__'